In [2]:
import pandas as pd
from os.path import join

In [3]:
data_path = "../Dataset-1/selfie_dataset.txt"#join("..", "..", "Dataset-1", "selfie_dataset.txt")
image_path = "../Dataset-1/images"#join("..", "..", "Dataset-1", "selfie_dataset.txt")#join("..", "..", "Dataset-1", "images")

In [4]:
headers = [
    "image_name", "score", "partial_faces" ,"is_female" ,"baby" ,"child" ,"teenager" ,"youth" ,"middle_age" ,"senior" ,"white" ,"black" ,"asian" ,"oval_face" ,"round_face" ,"heart_face" ,"smiling" ,"mouth_open" ,"frowning" ,"wearing_glasses" ,"wearing_sunglasses" ,"wearing_lipstick" ,"tongue_out" ,"duck_face" ,"black_hair" ,"blond_hair" ,"brown_hair" ,"red_hair" ,"curly_hair" ,"straight_hair" ,"braid_hair" ,"showing_cellphone" ,"using_earphone" ,"using_mirror", "braces" ,"wearing_hat" ,"harsh_lighting", "dim_lighting"
]
df_image_details = pd.read_csv(data_path, names=headers, delimiter=" ")
df_image_details.head(3)

,image_name,score,partial_faces,is_female,baby,child,teenager,youth,middle_age,senior,...,curly_hair,straight_hair,braid_hair,showing_cellphone,using_earphone,using_mirror,braces,wearing_hat,harsh_lighting,dim_lighting
0,00a454da495e11e28a7322000a1fa414_6,3.901,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,00cddb96ac4c11e3a30212279ba1b65f_6,4.385,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,01cdd7aa1a1a11e2aaa822000a1fb0dd_6,4.243,-1,1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [5]:
df_image_details = df_image_details[df_image_details.is_female != 0]
df_image_details.replace(to_replace=-1, value=0, inplace=True)

In [6]:
image_names = df_image_details.image_name.values.copy()
image_scores = df_image_details[headers[1]].values.copy()
image_attrs = df_image_details[headers[2:]].values.copy()

In [7]:
image_paths = [join(image_path, iname) + '.jpg' for iname in image_names]

In [8]:
image_paths_train, image_paths_test = image_paths[:-1000], image_paths[-1000:]
image_attrs_train, image_attrs_test = image_attrs[:-1000], image_attrs[-1000:]
image_scores_train, image_scores_test = image_scores[:-1000], image_scores[-1000:]

In [9]:
from keras.utils import Sequence
import numpy as np
import cv2

Using TensorFlow backend.


In [10]:
class ImageGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y1 = self.y[0][idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y2 = self.y[1][idx * self.batch_size:(idx + 1) * self.batch_size]

        # read your data here using the batch lists, batch_x and batch_y
        x = [(self.read_image(filename) - 127) / 255. for filename in batch_x] 
        y1 = [atrributes for atrributes in batch_y1]
        y2 = [atrributes for atrributes in batch_y2]
        return np.array(x), [np.array(y1), np.array(y2)]
    
    def read_image(self, fname):
        im = cv2.imread(fname)
        im = cv2.resize(im, (224, 224))
        return im;

# Training Model


In [23]:
from keras.applications import resnet50
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Model, load_model

In [12]:
model_rnet = load_model("resnet50.hdf5")

/home/npc/anaconda3/envs/py36/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [13]:
for layer in model_rnet.layers:
    layer.trainable = False

In [ ]:
model_classification = Dense(1000, activation='relu')(model_rnet.get_layer('avg_pool').output)
model_classification = Dense(36, activation='sigmoid', name='classification')(model_classification)

In [15]:
model_regression = Dense(128, activation='relu')(model_rnet.get_layer('avg_pool').output)
model_regression = Dense(1, name='regression')(model_regression)

In [16]:
model = Model(inputs=[model_rnet.input], outputs=[model_regression, model_classification])

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [24]:
model.compile(
    optimizer=Adam(0.0003),
    loss={
        'regression': 'mean_squared_error',
        'classification': 'binary_crossentropy'
    },
    metrics=[
        'accuracy'
    ]
)

In [19]:
train_gen = ImageGenerator(image_paths_train, (image_scores_train, image_attrs_train), batch_size=128)
test_gen = ImageGenerator(image_paths_test, (image_scores_test, image_attrs_test), batch_size=128)

In [20]:
train_len = len(image_paths_train)
test_len = len(image_paths_test)
train_len, test_len

(44227, 1000)

In [25]:
model.fit_generator(train_gen, validation_data=test_gen, epochs=200, 
                    steps_per_epoch=train_len // 256,
                   validation_steps=10, use_multiprocessing=False)

Epoch 1/200
 43/172 [======>.......................] - ETA: 1:16 - loss: 0.5347 - regression_loss: 0.2720 - classification_loss: 0.2627 - regression_acc: 1.8169e-04 - classification_acc: 0.8887

KeyboardInterrupt: 

In [26]:
model.save("chpt-4_1.hdf5")